# Problem 1: Generating Random Boolean Functions

The Deutsch–Jozsa algorithm is designed to work with functions that accept a fixed number of Boolean inputs and return a single Boolean output. Each function is guaranteed to be either constant (always returns False or always returns True) or balanced (returns True for exactly half of the possible input combinations).

**Purpose:** Generate a random boolean function with 4 inputs that is constant or balanced

**Fornula:**
Constant: `f(a,b,c,d) = k` where k ∈ {True, False} for all inputs
Balanced: `f(a,b,c,d) = True` for exactly 8 of 16 possible inputs

**Behaviour:** Returns a callable function that accepts 4 Boolean arguments and returns a boolean

## Solution
Will generate all 16 possible 4-bit Boolean input combinations using `itertools.product([False, True], repeat=4)`. The function will randomly choose between creating a constant or a balanced function. For the constantfunction, all inputs map to the same Boolean value. While for the balanced function, will use the `random.sample()` to randomly select 8 of the 16 inputs to return True, with the remaining 8 inputs returning False. The dictionary will store the input/output mapping, and then will return a closure function that performs O(1) lookups on the dictionary to determine the output for any input combination.

## Operations
`itertools.product([False, True], repeat=4)` generates all input combinations
`random.choice()` selects function type and constant value
`random.sample()` selects 8 random inputs for balanced functions
Dictionary comprehension creates the function mapping efficiently

## References
- IBM Quantum Learning - Deutsch-Jozsa Algorithm: https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa
- Python itertools documentation: https://docs.python.org/3/library/itertools.html
- Nielsen & Chuang, "Quantum Computation and Quantum Information", Section 1.4.3

In [1]:
# All the neccessary imports for my notebook
import random
from itertools import product
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt

## Helper Function: generate_inputs()

## Purpose
The helper function generates all the possible boolean input combinations for a given number of bits. It's used to create the complete input space for testing Boolean functions.

### Implementation
The helper function uses Python's 'itertools.product()' to create a Cartesian product of '[False, True]' repeated 'n_bits' times. This generates all the possible combinations without manual iteration.

### Example
For 4 bits:
- Total combinations: $2^4 = 16$
- Output ranges from '(False, False, False, False)' to '(True, True, True, True)'

In [2]:
# Helper function 
# Generates all possible input combinations
# Uses itertools.product to create a cartesian product of False or True repeated n times
# See https://docs.python.org/3/library/itertools.html#itertools.product
def generate_inputs(n_bits: int) -> list:
    """
    Generate all possible boolean inputs for n bits

    The function creates all possible tuples of boolean values of length n_bits
    
    Args
        : number of boolean input bits

    Returns:
      a list of tuples representing all possible input combinations
    """
    # Using itertools.product to generate all combinations
    # Product([False, True], repeat=n_bits) creates a cartesian product
    # Converting to list for easier indexing and manipulation
    return list(product([False, True], repeat=n_bits))

print("generate_inputs() helper function created")


generate_inputs() helper function created


In [3]:
# Testing the helper function
test_inputs = generate_inputs(4)
print(f"Number of possible inputs for 4 bits: {len(test_inputs)}")
print(f"First 5 input combinations: {test_inputs[:5]}")
print(f"Last 5 input combinations: {test_inputs[-5:]}")

Number of possible inputs for 4 bits: 16
First 5 input combinations: [(False, False, False, False), (False, False, False, True), (False, False, True, False), (False, False, True, True), (False, True, False, False)]
Last 5 input combinations: [(True, False, True, True), (True, True, False, False), (True, True, False, True), (True, True, True, False), (True, True, True, True)]


In [4]:
# Generates a random constant or balanced Boolean function with 4 inputs
# Returns a callable function that maps 4 Boolean inputs to a Boolean output
# See https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa
def random_constant_balanced() -> Callable:
    """
    Generating a random constant or balanced Boolean function with 4 inputs
    Formula: Constant returns same value for all 16 inputs, Balanced returns True for exactly 8/16 inputs
    
    The function randomly chooses between constant or balanced type
    For constant all 16 inputs map to same Boolean value (True or False)
    For balanced exactly 8 inputs map to True and the remaining 8 inputs map to False
    
    Returns:
        a callable function f(a,b,c,d) that accepts 4 Boolean arguments and returns Boolean output
    """
    # Generating all 16 possible input combinations using the helper function
    all_inputs = generate_inputs(4)
    
    # Randomly selects function type
    # random.choice() picks either constant or balanced
    function_type = random.choice(['constant', 'balanced'])
    
    # if random.choice picks constant all inputs map to the same value
    if function_type == 'constant':
        # Randomly choose True or False as the constant output
        constant_value = random.choice([True, False])
        # Dictionary comprehension maps all inputs to same constant value
        function_map = {inputs: constant_value for inputs in all_inputs}
        
    # if random.choice picks balanced exactly 8 inputs return true and the other 8 return false
    else:
        # random.sample() selects 8 unique random inputs without replacement
        true_inputs = random.sample(all_inputs, 8)
        # Dictionary comprehension maps selected inputs to True and the other inputs to False
        function_map = {inputs: (inputs in true_inputs) for inputs in all_inputs}
    
    # Returning a closure function that encapsulates the mapping
    def boolean_function(a: bool, b: bool, c: bool, d: bool) -> bool:
        """
        Boolean function with 4 inputs
        Performs O(1) dictionary lookup to determine output
        
        Args:
            a, b, c, d: Boolean input values
            
        Returns:
             a boolean output based on the generated function mapping
        """
        # Dictionary lookup using input tuple as key
        return function_map[(a, b, c, d)]
    
    # Stores metadata for testing
    boolean_function.function_type = function_type
    boolean_function.function_map = function_map
    
    return boolean_function

In [5]:
# Testing the random_constant_balanced() function
# Generating multiple function & verifying that they are eiter constant or balanced
print("Testing the random_constant_balanced() function")

# Testing with 6 different generated functions
for i in range(6):
    # Generating a random function
    f = random_constant_balanced()

    # Getting all the possible input combinations
    all_inputs = generate_inputs(4)

    # Counting how many inouts return true by evaluting function on all inputs
    # sum() counts the true values (True = 1, False = 0)
    true_count = sum(f(*inputs) for inputs in all_inputs)

    # Showing the function information
    print(f"\nFunction {i+1}:")
    print(f"Type: {f.function_type}")
    print(f"True count: {true_count}/16")

    # Verifying the correctness based on the function type
    if f.function_type == 'constant':
        # Constant functions should return True for all (16) or none (0)
        is_valid = (true_count == 0 or true_count == 16)
        print(f"Verification: {'Valid constant function' if is_valid else 'Invalid!'}")
    # Balanced functions
    else:
        # Balanced functions should return True for exactly 8 inputs
        is_valid = (true_count == 8)
        print(f"Verification: {'Valid balanced function' if is_valid else 'Invalid!'}")

    # Printing out outputs for first 4 input combinations
    print(f"Outputs:")
    for j in range(4):
        test_input = all_inputs[j]
        output = f(*test_input)
        print(f"f{test_input} = {output}") 

Testing the random_constant_balanced() function

Function 1:
Type: balanced
True count: 8/16
Verification: Valid balanced function
Outputs:
f(False, False, False, False) = False
f(False, False, False, True) = True
f(False, False, True, False) = False
f(False, False, True, True) = False

Function 2:
Type: balanced
True count: 8/16
Verification: Valid balanced function
Outputs:
f(False, False, False, False) = False
f(False, False, False, True) = False
f(False, False, True, False) = False
f(False, False, True, True) = False

Function 3:
Type: balanced
True count: 8/16
Verification: Valid balanced function
Outputs:
f(False, False, False, False) = True
f(False, False, False, True) = False
f(False, False, True, False) = True
f(False, False, True, True) = True

Function 4:
Type: constant
True count: 0/16
Verification: Valid constant function
Outputs:
f(False, False, False, False) = False
f(False, False, False, True) = False
f(False, False, True, False) = False
f(False, False, True, True) = Fa

# Problem 2: Classical Testing for Function Type

The Deutsch's algorithm is designed to demonstrate a potential advantage of quantum computing over classical computation. To understand this advantage, we must first understand the classical cost of solving the underlying problem.

 We must write a Python function
 `determine_constant_balanced` that takes as input a function `f`, as defined in Problem 1. The function should analyze `f` and return the string `"constant"` or `"balanced"` depending on whether the function is constant or balanced.


**Purpose:** Determines if a given Boolean function with 4 inputs is constant or balanced

**Formula:** Takes a function `f(a, b, c, d)` as the inputs

**Behaviour:** Returns a string either `"constant"` or `"balanced"`

## Solution

The function evaluates the input combinations and compares outputs. If two different outputs (one True, one False) are found, the function must be balanced. If the function is evaluated and found that more than half of the inputs (9 out of 16) and they're all the same, the function must be constant. Will generate all the possible inputs and check them sequentially until the type of function can be determined with certainty.

## Efficiency Analysis

**Maximum number of function calls needed:** 9 out of 16

- If the first 8 evaluations all return the same value, the function could still be balanced
- On the 9th evaluation, we are guaranteed certainty
- **Best case:** 2 evaluations, if the first two differ the function is balanced
- **Worst case:** 9 evaluations to confirm that the function is constant

## Operations

- `generate_inputs(4)` creates all 16 possible input combinations
- Loops through the inputs and evaluates the function `f(*inputs)` 
- Compares outputs to determine if the functions are constant or balanced
- Returns a string `"constant"` for constant function or `"balanced"` for balanced functions

## References

- IBM Quantum Learning - Deutsch's Algorithm: https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/deutsch-algorithm
- Nielsen & Chuang, "Quantum Computation and Quantum Information", Section 1.4.4

In [6]:
# Classical algorithm to determine if a function is constant or balanced
# Evaluates function on inputs until type can be determined with certainty
# See https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/deutsch-algorithm
def determine_constant_balanced(f: Callable) -> str:
    """
    Determines if a Boolean function with 4 inputs is constant or balanced
    Formula: f(a,b,c,d)
    
    Classical Testing approach evaluates the function on the inputs sequentially
    If two different outputs found the function is found to be balanced
    If 9+ same outputs found the function is found to be constant
    
     Args:
        f: callable function that accepts 4 Boolean arguments

     Returns:
        a string "constant" or "balanced"
    """
    # Generates all the 16 possible input combinations
    all_inputs = generate_inputs(4)
    
    # Evaluates the function on the first input to get the initial output
    first_output = f(*all_inputs[0])
    
    # Counter for the number of function evaluations
    evaluations = 1
    
    # Checking to see the remaining inputs for different output
    for i in range(1, len(all_inputs)):
        current_output = f(*all_inputs[i])
        evaluations += 1
        
        # If a different output is found, the function must be balanced
        if current_output != first_output:
            return "balanced"
        
        # If 9 inputs are found and they are all the same, the function must be constant
        # More than half of the 16 inputs confirms that the function is constant
        if evaluations >= 9:
            return "constant"
    
    # If all of the 16 inputs give the same output, the function is constant
    return "constant"

print("determine_constant_balanced() function created")

determine_constant_balanced() function created


## Tests for determine_constant_balanced()

The tests verify:

1. **Correctness of classification**
    - Generates random constant and balanced functions
    - Uses the classical algorithm to determine function type
    - Compares determined type with actual type from metadata
    - Demonstrates that the algorithm correctly identifies both function types

2. **Algorithm reliability**
    - Testing with 6 different random functions
    - Ensures consistent accurate classification
    - Verifies that the algorithm works with multiple test cases 

3. **Validation against ground truth**
    - Actual type stored in function metadata (function_type)
    - The determined type computed by classical algorithm
    - Direct comparison confirms algorithmic correctness
    - Provides clear pass/fail feedback for each test